<a href="https://colab.research.google.com/github/rohitd2209/TFX/blob/main/Breast_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
!pip install -U tfx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.8.2
TFX version: 1.8.0


In [ ]:
import os
#pipeline for schema generation
PIPELINE_NAME = "cancer-simple"
SCHEMA_PIPELINE_NAME = "cancer-tfdv-schema"

# Output directory to store artifacts generated from the pipeline
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
SCHEMA_PIPELINE_ROOT = os.path.join('pipelines', SCHEMA_PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SCHEMA_METADATA_PATH = os.path.join('metadata', SCHEMA_PIPELINE_NAME,
                                    'metadata.db')

# Output directory where created models from the pipeline will be exported
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='cancer_tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/rohitd2209/TFX/main/Breast_cancer.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

('/tmp/cancer_tfx-data384i_nox/data.csv',
 <http.client.HTTPMessage at 0x7f1f22928310>)

In [ ]:
!head {_data_filepath}

mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
17.99,10.38,122.8,1001,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189,FALSE
20.57,17.77,132.9,1326,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.0186,0.0134,0.01389,0.003532,24.99,23.41,158.8,1956,0.1238,0.1866,0.2416,0.186,0.275,0.08902,FALSE
19.69,21.25,130,1203,0.1096,0.1599,0.1974,0.1279,0.2069,0.05999,0.7456,0.7869,4.585

In [ ]:

!sed -i '/\bNA\b/d' {_data_filepath}
!head {_data_filepath}

mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
17.99,10.38,122.8,1001,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189,FALSE
20.57,17.77,132.9,1326,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.0186,0.0134,0.01389,0.003532,24.99,23.41,158.8,1956,0.1238,0.1866,0.2416,0.186,0.275,0.08902,FALSE
19.69,21.25,130,1203,0.1096,0.1599,0.1974,0.1279,0.2069,0.05999,0.7456,0.7869,4.585

In [ ]:
def _create_schema_pipeline(pipeline_name: str,
                            pipeline_root: str,
                            data_root: str,
                            metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline for schema generation."""
  # ExampleGen
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # StatisticsGen
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # SchemaGen
  schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  components = [
      example_gen,
      statistics_gen,
      schema_gen,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_schema_pipeline(
      pipeline_name=SCHEMA_PIPELINE_NAME,
      pipeline_root=SCHEMA_PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=SCHEMA_METADATA_PATH))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
executor_specs {
  key: "SchemaGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.schema_gen.executor.Executor"
    }
  }
}
executor_specs {
  key: "StatisticsGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.statistics_gen.executor.Executor"
      }
    }
  }
}
custom_driver_specs {
  key: "CsvExampleGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.example_gen.driver.FileBasedDriver"
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "metada

INFO:absl:Processing input csv data /tmp/cancer_tfx-data384i_nox/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Value type <class 'NoneType'> of key version in exec_properties is not supported, going to drop it
INFO:absl:Value type <class 'list'> of key _beam_pipeline_args in exec_properties is not supported, going to drop it
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 1 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "pipelines/cancer-tfdv-schema/CsvExampleGen/examples/1"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:122870,xor_checksum:1654749950,sum_checksum:1654749950"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "cancer-tfdv-schema:2022-06-09T04:45:53.910393:CsvExampleGen:examples:0"
  }
}
custom_properties {
  key: "span"
  value {
    int_v

In [ ]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_artifacts_dict(metadata, latest_execution.id,
                                          [metadata_store_pb2.Event.OUTPUT])

# Non-public APIs, just for showcase.
from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

In [ ]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    SCHEMA_METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  stat_gen_output = get_latest_artifacts(metadata_handler, SCHEMA_PIPELINE_NAME,
                                         'StatisticsGen')
  stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]

  schema_gen_output = get_latest_artifacts(metadata_handler,
                                           SCHEMA_PIPELINE_NAME, 'SchemaGen')
  schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]

INFO:absl:MetadataStore with DB connection initialized


In [ ]:
# docs-infra: no-execute
visualize_artifacts(stats_artifacts)

In [ ]:
visualize_artifacts(schema_artifacts)

,Type,Presence,Valency,Domain
Feature name,,,,
'area error',FLOAT,required,,-
'compactness error',FLOAT,required,,-
'concave points error',FLOAT,required,,-
'concavity error',FLOAT,required,,-
'fractal dimension error',FLOAT,required,,-
'mean area',FLOAT,required,,-
'mean compactness',FLOAT,required,,-
'mean concave points',FLOAT,required,,-
'mean concavity',FLOAT,required,,-


In [ ]:
import shutil

_schema_filename = 'schema.pbtxt'
SCHEMA_PATH = 'schema'

os.makedirs(SCHEMA_PATH, exist_ok=True)
_generated_path = os.path.join(schema_artifacts[0].uri, _schema_filename)

# Copy the 'schema.pbtxt' file from the artifact uri to a predefined path.
shutil.copy(_generated_path, SCHEMA_PATH)

'schema/schema.pbtxt'

In [ ]:
print(f'Schema at {SCHEMA_PATH}-----')
!cat {SCHEMA_PATH}/*

Schema at schema-----
feature {
  name: "area error"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "compactness error"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "concave points error"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "concavity error"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "fractal dimension error"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "mean area"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "mean compactness"
  type: FLOAT
  presence 

In [ ]:
_module_file = 'cancer_trainer.py'

In [ ]:
%%writefile {_module_file}


from typing import List, Text
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio


#      Transfrom component  
_FEATURE_KEYS = [
      "mean radius",'mean texture','mean perimeter','mean area','mean smoothness',
      'mean compactness','mean concavity','mean concave points','mean symmetry',
      'mean fractal dimension','radius error','texture error','perimeter error',
      'area error','smoothness error','compactness error','concavity error',
      'concave points error','symmetry error','fractal dimension error',
      'worst radius','worst texture','worst perimeter','worst area',
      'worst smoothness','worst compactness','worst concavity','worst concave points',
      'worst symmetry','worst fractal dimension'

]
_LABEL_KEY = 'target'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10


# NEW: TFX Transform will call this function.
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature.
  """
  outputs = {}

  # Uses features defined in _FEATURE_KEYS only.
  for key in _FEATURE_KEYS:
    # tft.scale_to_z_score computes the mean and variance of the given feature
    # and scales the output based on the result.
    outputs[key] = tft.scale_to_z_score(inputs[key])

  # For the label column we provide the mapping from string to index.
  # We could instead use `tft.compute_and_apply_vocabulary()` in order to
  # compute the vocabulary dynamically and perform a lookup.
  # Since in this example there are only 2 possible values, we use a hard-coded
  # table for simplicity.
  table_keys = ['FALSE', 'TRUE']
  initializer = tf.lookup.KeyValueTensorInitializer(
      keys=table_keys,
      values=tf.cast(tf.range(len(table_keys)), tf.float32),
      key_dtype=tf.string,
      value_dtype=tf.float32)
  table = tf.lookup.StaticHashTable(initializer, default_value=-1)
  outputs[_LABEL_KEY] = table.lookup(inputs[_LABEL_KEY])

  return outputs


# NEW: This function will apply the same transform operation to training data
#      and serving requests.
def _apply_preprocessing(raw_features, tft_layer):
  transformed_features = tft_layer(raw_features)
  if _LABEL_KEY in raw_features:
    transformed_label = transformed_features.pop(_LABEL_KEY)
    return transformed_features, transformed_label
  else:
    return transformed_features, None


# NEW: This function will create a handler function which gets a serialized
#      tf.example, preprocess and run an inference with it.
def _get_serve_tf_examples_fn(model, tf_transform_output):
  # We must save the tft_layer to the model to ensure its assets are kept and
  # tracked.
  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_examples):
    # Expected input is a string which is serialized tf.Example format.
    feature_spec = tf_transform_output.raw_feature_spec()
    # Because input schema includes unnecessary fields like 'species' and
    # 'island', we filter feature_spec to include required keys only.
    required_feature_spec = {
        k: v for k, v in feature_spec.items() if k in _FEATURE_KEYS
    }
    parsed_features = tf.io.parse_example(serialized_tf_examples,
                                          required_feature_spec)

    # Preprocess parsed input with transform operation defined in
    # preprocessing_fn().
    transformed_features, _ = _apply_preprocessing(parsed_features,
                                                   model.tft_layer)
    # Run inference with ML model.
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  dataset = data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(batch_size=batch_size),
      schema=tf_transform_output.raw_metadata.schema)

  transform_layer = tf_transform_output.transform_features_layer()
  def apply_transform(raw_features):
    return _apply_preprocessing(raw_features, transform_layer)

  return dataset.map(apply_transform).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying spaceship data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [
      keras.layers.Input(shape=(1,), name=key)
      for key in _FEATURE_KEYS
  ]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(2)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # NEW: Save a computation graph including transform layer.
  signatures = {
      'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Implements the spaceship pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')


########       ExampleValidator                               ##########
  # Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])


##########       Transform             #################
  # NEW: Transforms input data using preprocessing_fn in the 'module_file'.
  transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],
      materialize=False,
      module_file=module_file)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],

      # NEW: Pass transform_graph to the trainer.
      transform_graph=transform.outputs['transform_graph'],

      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      statistics_gen,
      schema_importer,
      example_validator,

      transform,  # NEW: Transform component was added to the pipeline.

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/content/cancer_trainer.py' (including modules: ['cancer_trainer']).
INFO:absl:User module package has hash fingerprint version ac3dd0b49a40b9c9323eb414a5f921f3580593bae107df6dd780ce9c043680ff.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmp_tlxna9c/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpichzb9w3', '--dist-dir', '/tmp/tmpho3wwz5o']
INFO:absl:Successfully built user code wheel distribution at 'pipelines/cancer-simple/_wheels/tfx_user_code_Transform-0.0+ac3dd0b49a40b9c9323eb414a5f921f3580593bae107df6dd780ce9c043680ff-py3-none-any.whl'; target user module is 'cancer_trainer'.
INFO:absl:Full user module path is 'cancer_trainer@pipelines/cancer-simple/_wheels/tfx_user_code_Transform-0.0+ac3dd0b49a40b9c9323eb414a5f921f3580593bae107df6dd780ce9c043680ff-py3-none-any.whl'
INFO:absl:Genera

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.
INFO:absl:Feature area error has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature compactness error has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature concave points error has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature concavity error has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature fractal dimension error has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mean area has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mean compactness has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mean concave points has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mean concavity has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mean fractal dimension has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mean perimeter has a 

INFO:tensorflow:Assets written to: pipelines/cancer-simple/Transform/transform_graph/4/.temp_path/tftransform_tmp/d75dcba685674bb38f39ce321c583ab7/assets


INFO:tensorflow:Assets written to: pipelines/cancer-simple/Transform/transform_graph/4/.temp_path/tftransform_tmp/d75dcba685674bb38f39ce321c583ab7/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Feature area error has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature compactness error has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature concave points error has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature concavity error has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature fractal dimension error has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mean area has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mean compactness has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mean concave points has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mean concavity has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mean fractal dimension has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mean perimeter ha

100/100 [==============================] - 2s 12ms/step - loss: 0.1558 - sparse_categorical_accuracy: 0.9235 - val_loss: 0.0344 - val_sparse_categorical_accuracy: 0.9800
INFO:tensorflow:Assets written to: pipelines/cancer-simple/Trainer/model/6/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/cancer-simple/Trainer/model/6/Format-Serving/assets
INFO:absl:Training complete. Model written to pipelines/cancer-simple/Trainer/model/6/Format-Serving. ModelRun written to pipelines/cancer-simple/Trainer/model_run/6
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 6 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model': [Artifact(artifact: uri: "pipelines/cancer-simple/Trainer/model/6"
custom_properties {
  key: "name"
  value {
    string_value: "cancer-simple:2022-06-09T04:46:26.161652:Trainer:model:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.8.0"
  }
}
name: "cancer-simple:2022-06-09T04:46:26.161652:Trainer:model:0"
, artifact_type: name: "Model"
base_type: MODEL
)], 'model_run': [Artifact(artifact: uri: "pipelines/cancer-simple/Trainer/model_run/6"
custom_properties {
  key: "name"
  value {
    string_

In [ ]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

serving_model/cancer-simple
serving_model/cancer-simple/1654750068
serving_model/cancer-simple/1654750068/variables
serving_model/cancer-simple/1654750068/variables/variables.data-00000-of-00001
serving_model/cancer-simple/1654750068/variables/variables.index
serving_model/cancer-simple/1654750068/keras_metadata.pb
serving_model/cancer-simple/1654750068/assets
serving_model/cancer-simple/1654750068/saved_model.pb


In [ ]:
!saved_model_cli show --dir {SERVING_MODEL_DIR}/$(ls -1 {SERVING_MODEL_DIR} | sort -nr | head -1) --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['examples'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_examples:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output_0'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: StatefulPartitionedCall_2:0
Method name is: tensorflow/serving/predict


In [ ]:
# Find a model with the latest timestamp.
model_dirs = (item for item in os.scandir(SERVING_MODEL_DIR) if item.is_dir())
model_path = max(model_dirs, key=lambda i: int(i.name)).path

loaded_model = tf.keras.models.load_model(model_path)
inference_fn = loaded_model.signatures['serving_default']

In [ ]:
# Prepare an example and run inference.
features = {

'mean radius' : tf.train.Feature(float_list=tf.train.FloatList(value=[13.08])),
'mean texture' : tf.train.Feature(float_list=tf.train.FloatList(value=[15.71])),
'mean perimeter' : tf.train.Feature(float_list=tf.train.FloatList(value=[85.63])),
'mean area' : tf.train.Feature(float_list=tf.train.FloatList(value=[520])),
'mean smoothness' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.1075])),
'mean compactness' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.127])),
'mean concavity' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.04568])),
'mean concave points' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.0311])),
'mean symmetry' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.1967])),
'mean fractal dimension' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.06811])),
'radius error' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.1852])),
'texture error' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.7477])),
'perimeter error' : tf.train.Feature(float_list=tf.train.FloatList(value=[1.383])),
'area error' : tf.train.Feature(float_list=tf.train.FloatList(value=[14.67])),
'smoothness error' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.004097])),
'compactness error' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.01898])),
'concavity error' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.01698])),
'concave points error' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.00649])),
'symmetry error' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.01678])),
'fractal dimension error' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.002425])),
'worst radius' : tf.train.Feature(float_list=tf.train.FloatList(value=[14.5])),
'worst texture' : tf.train.Feature(float_list=tf.train.FloatList(value=[20.49])),
'worst perimeter' : tf.train.Feature(float_list=tf.train.FloatList(value=[96.09])),
'worst area' : tf.train.Feature(float_list=tf.train.FloatList(value=[630.5])),
'worst smoothness' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.1312])),
'worst compactness' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.2776])),
'worst concavity' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.189])),
'worst concave points' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.07283])),
'worst symmetry' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.3184])),
'worst fractal dimension' : tf.train.Feature(float_list=tf.train.FloatList(value=[0.08183]))


}
example_proto = tf.train.Example(features=tf.train.Features(feature=features))
examples = example_proto.SerializeToString()

result = inference_fn(examples=tf.constant([examples]))
print(result['output_0'].numpy())

if result['output_0'].numpy()[0][1]>=0:
  print('Person having Cancer')
else:
  print('Person not having Cancer')

In [ ]:
# Prepare an example and run inference.
features = {

'mean radius': tf.train.Feature(float_list=tf.train.FloatList(value=[19.81])),
'mean texture': tf.train.Feature(float_list=tf.train.FloatList(value=[22.15])),
'mean perimeter': tf.train.Feature(float_list=tf.train.FloatList(value=[130.0])),
'mean area': tf.train.Feature(float_list=tf.train.FloatList(value=[1260.0])),
'mean smoothness': tf.train.Feature(float_list=tf.train.FloatList(value=[0.09831])),
'mean compactness': tf.train.Feature(float_list=tf.train.FloatList(value=[0.1027])),
'mean concavity': tf.train.Feature(float_list=tf.train.FloatList(value=[0.1479])),
'mean concave points': tf.train.Feature(float_list=tf.train.FloatList(value=[0.09498])),
'mean symmetry': tf.train.Feature(float_list=tf.train.FloatList(value=[0.1582])),
'mean fractal dimension': tf.train.Feature(float_list=tf.train.FloatList(value=[0.05395])),
'radius error': tf.train.Feature(float_list=tf.train.FloatList(value=[0.7582])),
'texture error': tf.train.Feature(float_list=tf.train.FloatList(value=[1.017])),
'perimeter error': tf.train.Feature(float_list=tf.train.FloatList(value=[5.865])),
'area error': tf.train.Feature(float_list=tf.train.FloatList(value=[112.4])),
'smoothness error': tf.train.Feature(float_list=tf.train.FloatList(value=[0.006494])),
'compactness error': tf.train.Feature(float_list=tf.train.FloatList(value=[0.01893])),
'concavity error': tf.train.Feature(float_list=tf.train.FloatList(value=[0.03391])),
'concave points error': tf.train.Feature(float_list=tf.train.FloatList(value=[0.01521])),
'symmetry error': tf.train.Feature(float_list=tf.train.FloatList(value=[0.01356])),
'fractal dimension error': tf.train.Feature(float_list=tf.train.FloatList(value=[0.001997])),
'worst radius': tf.train.Feature(float_list=tf.train.FloatList(value=[27.32])),
'worst texture': tf.train.Feature(float_list=tf.train.FloatList(value=[30.88])),
'worst perimeter': tf.train.Feature(float_list=tf.train.FloatList(value=[186.8])),
'worst area': tf.train.Feature(float_list=tf.train.FloatList(value=[2398.0])),
'worst smoothness': tf.train.Feature(float_list=tf.train.FloatList(value=[0.1512])),
'worst compactness': tf.train.Feature(float_list=tf.train.FloatList(value=[0.315])),
'worst concavity': tf.train.Feature(float_list=tf.train.FloatList(value=[0.5372])),
'worst concave points': tf.train.Feature(float_list=tf.train.FloatList(value=[0.2388])),
'worst symmetry': tf.train.Feature(float_list=tf.train.FloatList(value=[0.2768])),
'worst fractal dimension': tf.train.Feature(float_list=tf.train.FloatList(value=[0.07615]))


}
example_proto = tf.train.Example(features=tf.train.Features(feature=features))
examples = example_proto.SerializeToString()

result = inference_fn(examples=tf.constant([examples]))
print(result['output_0'].numpy())

if result['output_0'].numpy()[0][1]>=0:
  print('Person having Cancer')
else:
  print('Person not having Cancer')


[[13.315927  -4.1935344]]
Person not having Cancer
